In [2]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('income_train.csv')
test_data = pd.read_csv('income_test.csv')

# manipulate the Catetgory column 
# set >50K as +1; <=50K as -1
train_data['Category'] = train_data['Category'].apply(lambda x : +1 if x=='>50K' else -1)

# exclude the Id column from features dataframe
train_data = pd.get_dummies(train_data)
features = list(train_data.columns[train_data.columns != 'Id'])
features.remove('Category')

# # check if the data is balance
# greater_than_50 = train_data[train_data['Category'] == 1]
# less_than_50 = train_data[train_data['Category'] == -1]

# # the data is imbalance; tried to throw away data to balance the data
# ratio = len(greater_than_50) / len(less_than_50)
# less_than_50 = less_than_50.sample(frac = ratio, random_state=1)
# train_data_new = greater_than_50.append(less_than_50)

# count_greater = (train_data_new['Category'] == 1).sum()
# count_less = (train_data_new['Category'] == -1).sum()


FileNotFoundError: [Errno 2] File b'income_train.csv' does not exist: b'income_train.csv'

In [3]:
# Decision Tree 
# Find the fit parameters 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

min_samples_leaf = [1,10,50,100,200,300]
max_depth = [1,5,10,15,20]
hyperparameters = {'min_samples_leaf':[1, 10, 50, 100, 200, 300],
                   'max_depth':[1, 5, 10, 15, 20]}

search = GridSearchCV(estimator=DecisionTreeClassifier(min_samples_leaf=...,
                                                       max_depth=...,
                                                       random_state=1),
                      param_grid=hyperparameters,
                      cv=6,
                      return_train_score=True)

best_param = search.fit(train_data[features], train_data['Category']).best_params_


NameError: name 'train_data' is not defined

In [4]:
# Model Training 
decision_tree_model = DecisionTreeClassifier(max_depth=best_param['max_depth'],
                                             min_samples_leaf=best_param['min_samples_leaf'])
decision_tree_model.fit(train_data[features], train_data['Category'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [5]:
# Decision Tree Predict
test_data = pd.get_dummies(test_data)
decision_tree_prediction = decision_tree_model.predict(test_data[features])
predictions = []
for i in range(len(decision_tree_prediction)):
    if decision_tree_prediction[i] == 1:
        predictions.append('>50K')
    else: 
        predictions.append('<=50K')

to_save = test_data[['Id']].copy()
to_save.loc[:, 'Category'] = predictions
to_save.to_csv('submission_decision_tree_(not balanced).csv', index=False)
